<a href="https://colab.research.google.com/github/inyunita/Data-Science-Case-Study/blob/main/K_Means_World_Education_(Data_Form_Big_Query).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer

In [1]:
# Install and import the required libraries
!pip install google-cloud-bigquery
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

In [3]:
# Authenticate with your Google account to access BigQuery
auth.authenticate_user()

In [11]:
# Set your project ID
project_id = 'homework4-394107'

In [12]:
# Create a BigQuery client
client = bigquery.Client(project=project_id)

In [15]:
# Set the BigQuery dataset and table name
dataset_name = 'education_pivot'
table_name = 'education_pivot'

In [16]:
# Construct the SQL query to select all data from the table
query = f"SELECT * FROM `{project_id}.{dataset_name}.{table_name}`"

# Execute the query and store the results in a Pandas DataFrame
df = client.query(query).to_dataframe()


In [17]:
df

,country_name,year,GDP_Education,LitteracyRate_Adult,LiteracyRate_Youth,pupil_teacher_preprimary,pupil_teacher_primary,pupil_teacher_secondary,pupil_teacher_tertiary
0,Afghanistan,2011,3.46201,31.448851,46.990051,NaN,43.51915,NaN,31.89532
1,Albania,2011,NaN,96.845299,98.791191,18.87546,19.86917,15.24414,13.79110
2,Albania,2012,NaN,97.246971,99.231400,18.50724,19.48298,14.89293,NaN
3,Albania,2018,NaN,98.141151,99.330002,17.09409,17.57287,11.19927,14.06519
4,Algeria,2018,5.86635,81.407837,97.426521,NaN,24.32018,NaN,27.29340
...,...,...,...,...,...,...,...,...,...
2315,Zimbabwe,2010,1.54406,NaN,NaN,NaN,NaN,NaN,23.18329
2316,Zimbabwe,2015,5.81279,NaN,NaN,NaN,NaN,NaN,17.93083
2317,Zimbabwe,2016,5.47262,NaN,NaN,NaN,NaN,NaN,NaN
2318,Zimbabwe,2017,5.38106,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2320 entries, 0 to 2319
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   country_name              2320 non-null   object 
 1   year                      2320 non-null   Int64  
 2   GDP_Education             1844 non-null   float64
 3   LitteracyRate_Adult       896 non-null    float64
 4   LiteracyRate_Youth        879 non-null    float64
 5   pupil_teacher_preprimary  1341 non-null   float64
 6   pupil_teacher_primary     1624 non-null   float64
 7   pupil_teacher_secondary   1337 non-null   float64
 8   pupil_teacher_tertiary    846 non-null    float64
dtypes: Int64(1), float64(7), object(1)
memory usage: 165.5+ KB


In [19]:
df.shape

(2320, 9)

In [20]:
df['country_name'].nunique()

254

In [23]:
#numerical column
numerical_col = df.select_dtypes(include=[np.number])
numerical_col.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
year,2320.0,2014.553879,2.926089,2010.0,2012.0,2015.0,2017.0,2020.0
GDP_Education,1844.0,4.391219,1.67724,0.78744,3.307505,4.121038,5.23605,13.51266
LitteracyRate_Adult,896.0,82.881692,16.041018,22.31155,71.695152,88.998039,95.589722,99.999947
LiteracyRate_Youth,879.0,90.173539,12.250694,30.791611,85.540943,96.083893,99.008144,99.999962
pupil_teacher_preprimary,1341.0,19.096851,9.078666,4.76812,13.08992,17.65605,22.68402,113.9653
pupil_teacher_primary,1624.0,23.559999,12.181403,6.13353,14.28091,20.201505,30.236798,84.32027
pupil_teacher_secondary,1337.0,16.649709,7.353111,4.97932,11.29297,15.08599,20.52323,72.31002
pupil_teacher_tertiary,846.0,16.726051,8.669146,1.16169,10.71786,15.25877,19.95643,64.40642


In [25]:
df.isna().sum()

country_name                   0
year                           0
GDP_Education                476
LitteracyRate_Adult         1424
LiteracyRate_Youth          1441
pupil_teacher_preprimary     979
pupil_teacher_primary        696
pupil_teacher_secondary      983
pupil_teacher_tertiary      1474
dtype: int64

In [26]:
numeric_cols = df.select_dtypes(include='number').columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

In [28]:
df.isna().sum()

country_name                0
year                        0
GDP_Education               0
LitteracyRate_Adult         0
LiteracyRate_Youth          0
pupil_teacher_preprimary    0
pupil_teacher_primary       0
pupil_teacher_secondary     0
pupil_teacher_tertiary      0
dtype: int64

In [29]:
df

,country_name,year,GDP_Education,LitteracyRate_Adult,LiteracyRate_Youth,pupil_teacher_preprimary,pupil_teacher_primary,pupil_teacher_secondary,pupil_teacher_tertiary
0,Afghanistan,2011,3.462010,31.448851,46.990051,17.65605,43.519150,15.08599,31.89532
1,Albania,2011,4.121038,96.845299,98.791191,18.87546,19.869170,15.24414,13.79110
2,Albania,2012,4.121038,97.246971,99.231400,18.50724,19.482980,14.89293,15.25877
3,Albania,2018,4.121038,98.141151,99.330002,17.09409,17.572870,11.19927,14.06519
4,Algeria,2018,5.866350,81.407837,97.426521,17.65605,24.320180,15.08599,27.29340
...,...,...,...,...,...,...,...,...,...
2315,Zimbabwe,2010,1.544060,88.998039,96.083893,17.65605,20.201505,15.08599,23.18329
2316,Zimbabwe,2015,5.812790,88.998039,96.083893,17.65605,20.201505,15.08599,17.93083
2317,Zimbabwe,2016,5.472620,88.998039,96.083893,17.65605,20.201505,15.08599,15.25877
2318,Zimbabwe,2017,5.381060,88.998039,96.083893,17.65605,20.201505,15.08599,15.25877


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
# Save DataFrame to CSV file
file_name = '/content/drive/MyDrive/Generasi Gigih/Task/Dataset/education_clean.csv'
df.to_csv(file_name, index=False)

In [33]:
numeric_cols

Index(['year', 'GDP_Education', 'LitteracyRate_Adult', 'LiteracyRate_Youth',
       'pupil_teacher_preprimary', 'pupil_teacher_primary',
       'pupil_teacher_secondary', 'pupil_teacher_tertiary'],
      dtype='object')

In [34]:
col_numeric_feature = numerical_col.columns.tolist()
col_numeric_feature

['year',
 'GDP_Education',
 'LitteracyRate_Adult',
 'LiteracyRate_Youth',
 'pupil_teacher_preprimary',
 'pupil_teacher_primary',
 'pupil_teacher_secondary',
 'pupil_teacher_tertiary']

In [38]:
attribute = df[col_numeric_feature]
scaler = MinMaxScaler()
#fit transform
scaled = scaler.fit_transform(attribute)

In [39]:
scaled

array([[0.1       , 0.21017868, 0.11761474, ..., 0.47815806, 0.15010493,
        0.48594768],
       [0.1       , 0.26196778, 0.95939358, ..., 0.17567736, 0.15245378,
        0.1996911 ],
       [0.2       , 0.26196778, 0.96456388, ..., 0.17073803, 0.14723759,
        0.22289731],
       ...,
       [0.6       , 0.36818067, 0.85838416, ..., 0.17992789, 0.15010493,
        0.22289731],
       [0.7       , 0.36098552, 0.85838416, ..., 0.17992789, 0.15010493,
        0.22289731],
       [0.8       , 0.22002292, 0.85838416, ..., 0.17992789, 0.15010493,
        0.22289731]])

In [44]:
df_scaled = pd.DataFrame(scaled)
df_scaled.columns = col_numeric_feature
df_scaled = df_scaled.drop('year', axis=1)

In [45]:
df_scaled

,GDP_Education,LitteracyRate_Adult,LiteracyRate_Youth,pupil_teacher_preprimary,pupil_teacher_primary,pupil_teacher_secondary,pupil_teacher_tertiary
0,0.210179,0.117615,0.234053,0.118024,0.478158,0.150105,0.485948
1,0.261968,0.959394,0.982534,0.129191,0.175677,0.152454,0.199691
2,0.261968,0.964564,0.988895,0.125819,0.170738,0.147238,0.222897
3,0.261968,0.976074,0.990320,0.112878,0.146308,0.092379,0.204025
4,0.399122,0.760684,0.962816,0.118024,0.232605,0.150105,0.413184
...,...,...,...,...,...,...,...
2315,0.059458,0.858384,0.943416,0.118024,0.179928,0.150105,0.348197
2316,0.394913,0.858384,0.943416,0.118024,0.179928,0.150105,0.265147
2317,0.368181,0.858384,0.943416,0.118024,0.179928,0.150105,0.222897
2318,0.360986,0.858384,0.943416,0.118024,0.179928,0.150105,0.222897


In [46]:
K_value = []
sil_score_kmeans=[]
# DBI_score_kmeans=[]

for kluster in range (2,11):
  kmeans = KMeans(n_clusters=kluster, n_init = 10)
  K_value.append(kluster)
  kmeans.fit(df_scaled)
  from sklearn.metrics import silhouette_score
  score = silhouette_score(df_scaled, kmeans.labels_)
  sil_score_kmeans.append(score)


In [47]:
tbl_sil_kmeans = pd.DataFrame({'Nilai K': K_value, 'Silhouette Score' : sil_score_kmeans})
tbl_sil_kmeans

,Nilai K,Silhouette Score
0,2,0.552772
1,3,0.437347
2,4,0.288347
3,5,0.323836
4,6,0.315144
5,7,0.252329
6,8,0.219224
7,9,0.228082
8,10,0.239983


In [48]:
kmeans_2 = KMeans(n_clusters=3, n_init = 10)
kmeans_2.fit(df_scaled)
df_kmeans = df.copy()
df_kmeans['Category'] = kmeans_2.labels_
df_kmeans


,country_name,year,GDP_Education,LitteracyRate_Adult,LiteracyRate_Youth,pupil_teacher_preprimary,pupil_teacher_primary,pupil_teacher_secondary,pupil_teacher_tertiary,Category
0,Afghanistan,2011,3.462010,31.448851,46.990051,17.65605,43.519150,15.08599,31.89532,1
1,Albania,2011,4.121038,96.845299,98.791191,18.87546,19.869170,15.24414,13.79110,0
2,Albania,2012,4.121038,97.246971,99.231400,18.50724,19.482980,14.89293,15.25877,0
3,Albania,2018,4.121038,98.141151,99.330002,17.09409,17.572870,11.19927,14.06519,0
4,Algeria,2018,5.866350,81.407837,97.426521,17.65605,24.320180,15.08599,27.29340,0
...,...,...,...,...,...,...,...,...,...,...
2315,Zimbabwe,2010,1.544060,88.998039,96.083893,17.65605,20.201505,15.08599,23.18329,0
2316,Zimbabwe,2015,5.812790,88.998039,96.083893,17.65605,20.201505,15.08599,17.93083,0
2317,Zimbabwe,2016,5.472620,88.998039,96.083893,17.65605,20.201505,15.08599,15.25877,0
2318,Zimbabwe,2017,5.381060,88.998039,96.083893,17.65605,20.201505,15.08599,15.25877,0


In [49]:
df.mean()

<ipython-input-49-c61f0c8f89b5>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.mean()


year                        2014.553879
GDP_Education                  4.335785
LitteracyRate_Adult           86.635864
LiteracyRate_Youth            93.844582
pupil_teacher_preprimary      18.488858
pupil_teacher_primary         22.552451
pupil_teacher_secondary       15.987150
pupil_teacher_tertiary        15.793822
dtype: float64

In [50]:
df.max()

country_name                 Zimbabwe
year                             2020
GDP_Education                13.51266
LitteracyRate_Adult         99.999947
LiteracyRate_Youth          99.999962
pupil_teacher_preprimary     113.9653
pupil_teacher_primary        84.32027
pupil_teacher_secondary      72.31002
pupil_teacher_tertiary       64.40642
dtype: object

In [51]:
df.min()

country_name                Afghanistan
year                               2010
GDP_Education                   0.78744
LitteracyRate_Adult            22.31155
LiteracyRate_Youth            30.791611
pupil_teacher_preprimary        4.76812
pupil_teacher_primary           6.13353
pupil_teacher_secondary         4.97932
pupil_teacher_tertiary          1.16169
dtype: object

In [54]:
cluster_0 = df_kmeans[df_kmeans['Category']==0]
cluster_0.mean()

<ipython-input-54-cd72449c7982>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cluster_0.mean()


year                        2014.668132
GDP_Education                  4.507221
LitteracyRate_Adult           89.843166
LiteracyRate_Youth            96.232415
pupil_teacher_preprimary      16.810234
pupil_teacher_primary         18.368298
pupil_teacher_secondary       14.221011
pupil_teacher_tertiary        15.246385
Category                       0.000000
dtype: float64

In [53]:
cluster_1 = df_kmeans[df_kmeans['Category']==1]
cluster_1.mean()

<ipython-input-53-b7f46f178604>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cluster_1.mean()


year                        2014.404762
GDP_Education                  3.624263
LitteracyRate_Adult           58.592491
LiteracyRate_Youth            71.658734
pupil_teacher_preprimary      24.197049
pupil_teacher_primary         35.428780
pupil_teacher_secondary       20.929827
pupil_teacher_tertiary        16.156999
Category                       1.000000
dtype: float64

In [59]:
# Replace 0 with "Good" and 1 with "Bad" in the 'Category' column
df_kmeans['Category'] = df_kmeans['Category'].replace({0: 'Good', 1: 'Bad'})

In [60]:
df_kmeans

,country_name,year,GDP_Education,LitteracyRate_Adult,LiteracyRate_Youth,pupil_teacher_preprimary,pupil_teacher_primary,pupil_teacher_secondary,pupil_teacher_tertiary,Category
0,Afghanistan,2011,3.462010,31.448851,46.990051,17.65605,43.519150,15.08599,31.89532,Bad
1,Albania,2011,4.121038,96.845299,98.791191,18.87546,19.869170,15.24414,13.79110,Good
2,Albania,2012,4.121038,97.246971,99.231400,18.50724,19.482980,14.89293,15.25877,Good
3,Albania,2018,4.121038,98.141151,99.330002,17.09409,17.572870,11.19927,14.06519,Good
4,Algeria,2018,5.866350,81.407837,97.426521,17.65605,24.320180,15.08599,27.29340,Good
...,...,...,...,...,...,...,...,...,...,...
2315,Zimbabwe,2010,1.544060,88.998039,96.083893,17.65605,20.201505,15.08599,23.18329,Good
2316,Zimbabwe,2015,5.812790,88.998039,96.083893,17.65605,20.201505,15.08599,17.93083,Good
2317,Zimbabwe,2016,5.472620,88.998039,96.083893,17.65605,20.201505,15.08599,15.25877,Good
2318,Zimbabwe,2017,5.381060,88.998039,96.083893,17.65605,20.201505,15.08599,15.25877,Good


In [61]:
# Save DataFrame to CSV file
file_name = '/content/drive/MyDrive/Generasi Gigih/Task/Dataset/education_clustered.csv'
df.to_csv(file_name, index=False)